In [1]:
import sys, pygame
from pygame import Color, surfarray
import numpy as np
import random
import cv2
import os

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
display_width = 900
display_height = 900

cell_size = 20 # Number of pixels per cell

cell_width = display_width//cell_size
cell_height = display_height//cell_size

display_width = cell_width*cell_size
display_height = cell_height*cell_size

color_dict = {
        0: (255, 255, 255),
        1: (0, 0, 0)
        }

n_bits = 3

EVOLVE_COLOR = True
SAVE_TO_DISK = False
#RULE_MODE = 'random' # 'iterate'= iterate through each rule, 'random'=random rule, integer rule # = single rule, list of #s= random numbers from list
#RULE_MODE = [30, 54, 60, 62, 90, 94, 102, 110, 122, 126, 150, 158, 182, 188, 190, 220, 222, 250]
RULE_MODE = 30

INIT_MODE = 'single' # 'single'=centered single cell, 'random'=random 0 or 1 for each cell in row

In [3]:
def init_states(itype='random', return_btm=False):
    states = [[0]*cell_width for i in range(cell_height)] # Zero State
    if itype == 'random':
        states[0] = [random.randrange(2) for i in range(cell_width)] # Random State
    elif itype == 'single':
        states[0] = [0]*cell_width
        states[0][cell_width//2] = 1
    if return_btm:
        return states[0]
    return states

In [4]:
def next_gen(left, middle, right, nth_rule):
    left, middle, right = int(bool(left)), int(bool(middle)), int(bool(right))
    value = int(str(left) + str(middle) + str(right), 2)
    return get_rule(nth_rule)[value]

def new_state(states, nth_rule=30):
    new = states[0].copy() # Get bottom row
    
    if nth_rule == 'random':
        nth_rule = random.randrange(2**(2**n_bits))
    
    new[1:-1] = [next_gen(new[i-1], new[i], new[i+1], nth_rule) for i in range(1, len(new)-1)] # New row, keep edge values
    
    # Set edge cells
    edge = int(np.round(sum(new)/len(new)))
    new[0], new[-1] = edge, edge
    
    # Iterate FIFO
    states.insert(0, new) # Add new row to state array
    states.pop() # Shift states up 1 row
    
    return states # return state array

"""
    Returns RGB list from state list
    - states: states object
    - colors: None=All pixels updated, colors=use prev colors object and only update btm row
"""
def states_to_pix(states, colors=None):
    if colors == None:
        return [[color_dict[c] for c in s] for s in states] # All state colors will match current color dict mapping
    else:
        btm = states[0] # Bottom
        
        # Add new row to color list with current color & shift FIFO
        colors.insert(0, [color_dict[s] for s in btm])
        colors.pop()
        return colors # Use all colors from colors, but update bottom row with new colors

In [5]:
def get_rule(nth_rule, n_bits=3):
    n_states = 2**n_bits
    rule = format(nth_rule, '0'+str(n_states)+'b')
    return np.array([int(b) for b in rule])[::-1]

In [6]:
def init_color_dict(color='random'):
    if color == 'random':
        color_dict[1] = (random.randrange(255), random.randrange(255), random.randrange(255))
    else:
        color_dict[1] = (0, 0, 0)

def update_color_dict(color='random', max_delta=10):
    if color == 'random':
        rgb = random.randrange(3) # Random Channel
        delta = random.randrange(-max_delta, max_delta) # Random change in color
        
        channels = list(color_dict[1])
        channel = channels[rgb]     
        channel += delta
        channel = channel if channel >= 0 else 0
        channel = channel if channel <= 255 else 255
        channels[rgb] = channel
        color_dict[1] = tuple(channels)

In [7]:
def save_frame(img, cnt):
    fname = format(f'{cnt:08d}.jpg')
    img = np.array(img)
    img = np.rot90(img, 3)
    img = np.flip(img, axis=1)
    img = cv2.imwrite(os.path.join('Screenshots', fname), img)
    
def save_screenshots_to_video(filename='1D_Cellular_Automata.avi', framerate=60, startframe=30, stopframe=300):
    video = []
    for i, file in enumerate(os.listdir('Screenshots')):
        if i < startframe:
            continue
        elif i > stopframe:
            break
        
        if file.endswith('.jpg'):
            img = cv2.imread(os.path.join('Screenshots', file))
            height, width, layers = img.shape
            size = (width, height)
            video += [img]

    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'DIVX'), framerate, size)
    for frame in video:
        out.write(frame)
    out.release()

### Overview:
- High Level Overview (Updating Pygame Surface):
  - Create surface from cell world resolution (Different from display resolution).
  - Copy surface to PixelArray object
  - Update PixelArray object from color list
  - color list gets colors from state list states
- states list:
  - the states list holds discrete boolean values, not RGB colors.
  - Bottom state starts at 0 for States obj, Top at -1. States are reversed when read to the screen.
- colors list:
  - the colors list holds rows of RGB tuples.
  - colors are translated from the states list using the color_dict.

In [11]:
states = init_states(itype=INIT_MODE) # Initialize States

init_color_dict() # Start with random color for on state
colors = states_to_pix(states) # Initialize colors

pygame.init()

screen = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('1D Cellular Automata')

#create a surface with the size as the array
surface = pygame.Surface((cell_width, cell_height))
surface = pygame.transform.scale(surface, (display_width, display_height)) # Scale Size Up


run = True
cnt = 0
nth_rule = 0
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False

    pygame.time.delay(30)
    
    # Reset bottom row if limited complexity
    if sum(states[0])/len(states[0]) < 0.05:
        states[0] = init_states(return_btm=True)
    
    if cnt % cell_height == cell_height-1 or cnt == 0:
        if RULE_MODE == 'random':
            nth_rule = random.randrange(2**(2**n_bits))
        elif RULE_MODE == 'iterate' and cnt > 0:
            nth_rule += 1
        elif type(RULE_MODE) == int:
            nth_rule = RULE_MODE
        elif type(RULE_MODE) == list:
            nth_rule = RULE_MODE[random.randrange(len(RULE_MODE))]
        
        # Reset States
        if type(RULE_MODE) != int:
            states = init_states(itype=INIT_MODE) # Initialize States
            
        print('Rule:', nth_rule)
    
    # Scale Screen Size Down to edit pixel array at cell size=1 --------------------------------------------------
    surface = pygame.transform.scale(surface, (cell_width, cell_height))
    px = pygame.PixelArray(surface)
    
    # Translate States to Surface as RGB Colors
    states = new_state(states, nth_rule=nth_rule)
    colors = states_to_pix(states, colors=colors)

    for i, r in enumerate(colors):
        px[:,i] = colors[len(colors) - 1 - i]

    px.close()
    
    # Scale Screen Size Up to display pixel array at cell size=multiplier ----------------------------------------
    surface = pygame.transform.scale(surface, (display_width, display_height))  # Scale Size Up     
    screen.blit(surface, (0, 0)) # Update all pixels on screen object
    
    if SAVE_TO_DISK:
        save_frame(surfarray.array3d(surface), cnt)
    
    pygame.display.update()
    
    if EVOLVE_COLOR:
        update_color_dict(max_delta=40)
        
    if cnt > 2000:
        break
    cnt += 1
    
pygame.quit()

Rule: 30
Rule: 30
Rule: 30
Rule: 30
Rule: 30


In [154]:
save_screenshots_to_video(filename='gif.avi', framerate=30, startframe=90, stopframe=300) # Run if screenshots already saved to disk